In [13]:
abnormal_files = ['Fall_while_standing_and_turning1', 'Fall_while_preparing_meal1', 'Walk_with_memory_loss5', 'Fall_while_climbing_at_somewhere_height1', 'Fall_while_during_getting_up_or_rising1', 'Walk_with_memory_loss6', 'Fall_while_standing_at_somewhere_height1', 'Fall_while_preparing_meal2', 'Walk_with_memory_loss7', 'Fall_while_walking_forward1', 'Fall_while_standing_quietly1', 'Run_with_disorientation', 'Fall_sideways_while_walking_forward1', 'Fall_while_initiation_of_walking2', 'Walk_with_memory_loss8', 'Fall_while_sitting_down_or_lowering1', 'Fall_in_bathroom1', 'Fall_backward_while_walking_and_turning1', 'Stand_on_coffee_table1', 'Fall_while_standing_and_reaching1', 'Fall_while_sitting_down1', 'Fall_while_during_getting_up_or_rising2', 'Fall_while_initiation_of_walking1']

In [14]:
def compress_list(lst):
    for i in range(0, len(lst)):
        lst[i][0] = lst[i][0].replace("find","walk")

    compressed_lst = []
    for i in range(0, len(lst)):
        if i == 0 or lst[i] != compressed_lst[-1] or "walk" not in lst[i]:
            compressed_lst.append(lst[i])
    return compressed_lst

In [16]:
import json

with open("generated_kgrc_data/script_scene1.json", "r") as json_file: #fill the path of the generated data file
    result_scripts = json.load(json_file)

with open("correct_kgrc_data/correct_scene1.json", "r") as json_file: #fill the path of the correct data file
    correct_data = json.load(json_file)

precision_sum = 0
recall_sum = 0
abnormal_count = 0

for f_num, correct in correct_data.items():
    if f_num not in abnormal_files:

        evaluation_data = result_scripts.get(f_num)
    
        for i, data in enumerate(evaluation_data):
        
            data = data.split()
            new_data = [str(data[1])[1:-1]]

            if len(data) == 4:
                new_data.append(str(data[2])[1:-1]) 
            elif len(data) == 6:
                new_data.append(str(data[2])[1:-1])
                new_data.append(str(data[4])[1:-1])

            evaluation_data[i] = new_data

        evaluation_data = compress_list(evaluation_data)

        for i, data in enumerate(correct):

            data = data.split()
            new_data = [str(data[0])[1:-1].lower()]

            if len(data) == 3:
                new_data.append(str(data[1])[1:-1]) 
            elif len(data) == 5:
                new_data.append(str(data[1])[1:-1])
                new_data.append(str(data[3])[1:-1])

            correct[i] = new_data

        correct = compress_list(correct)
    
        correct_len = len(correct)
        evaluation_len = len(evaluation_data)

        correct_num = 0
        for e_data in evaluation_data:
            for i, c_data in enumerate(correct):
                if e_data[0] == c_data[0]:

                    if len(e_data) == 1:
                        correct_num += 1
                        del correct[i]
                        break

                    elif len(e_data) == 2:
                        if e_data[1] == c_data[1]:
                            correct_num += 1
                            del correct[i]
                            break

                    else:
                        if e_data[1] == c_data[1] and e_data[2] == c_data[2]:
                            correct_num += 1
                            del correct[i]
                            break
    
        if evaluation_len == 0:
            precision = 0
        else:
            precision = correct_num/evaluation_len
    
        if correct_len == 0:
            recall = 0
        else:
            recall = correct_num/correct_len

        precision_sum += precision
        recall_sum += recall

    else:
        abnormal_count += 1

num = len(correct_data)-abnormal_count

print("precision: {:.3f}".format(precision_sum/num))
print("recall: {:.3f}".format(recall_sum/num))      
print("f1: {:.3f}".format(2*(precision_sum/num)*(recall_sum/num)/((precision_sum/num)+(recall_sum/num))) )

precision: 0.591
recall: 0.481
f1: 0.530
